# Arize AX: Improving Classification with LLMs using Prompt Learning

<img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="600">

In this notebook we will leverage the PromptLearningOptimizer developed here at Arize to improve upon the accuracy of LLMs on classification tasks. Specifically we will be classifying support queries into 30 different classes, including

Account Creation

Login Issues

Password Reset

Two-Factor Authentication

Profile Updates

Billing Inquiry

Refund Request

and 24 more. 

You can view the dataset in datasets/support_queries.csv.

**Note: This notebook `arizeax_support_query_classification.ipynb` complements `support_query_classification.ipynb` by using Arize Phoenix datasets, experiments, and prompt management for Prompt Learning. It's a more end to end way for you to visualize your iterative prompt improvement and see how it performs on train/test sets, and also leverages methods for advanced features.**

In [1]:
%pip install -q arize-phoenix openai pandas arize gql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys, os, getpass
import openai
import pandas as pd
from openai import AsyncOpenAI
import re
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
openai_client = AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])
ARIZE_API_KEY = getpass.getpass('ARIZE_API_KEY')
ARIZE_SPACE_ID = getpass.getpass('ARIZE_SPACE_ID')

In [4]:
# Add parent directory to path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# **Setup**

In [5]:
from arize.experimental.datasets import ArizeDatasetsClient
import pandas as pd

# add your Arize API key here
client = ArizeDatasetsClient(api_key=ARIZE_API_KEY)

## **Make train/test sets**

We use an 80/20 train/test split to train our prompt. The optimizer will use the training set to visualize and analyze its errors and successes, and make prompt updates based on these results. We will then test on the test set to see how that prompt performs on unseen data. 

We will be exporting these datasets to Arize AX. In Arize you will be able to view the experiments we run on the train/test sets.

In [6]:
import pandas as pd
from uuid import uuid1
from arize.experimental.datasets.utils.constants import GENERATIVE


data = pd.read_csv("../datasets/support_queries.csv")


#train_set = data.sample(frac=0.8, random_state=42).sample(frac=0.02) #small sample test
train_set = data.sample(frac=0.8, random_state=42) #full dataset test train 123 / test 31 examples
test_set = data.drop(train_set.index)

UUID = str(uuid1())[:8] #associate UUID with datasets and prompt name to track across runs
TRAIN_DATASET_NAME = "prompt_optimizer_training_data+" + UUID
TEST_DATASET_NAME = "prompt_optimizer_test_data+" + UUID

train_dataset_id = client.create_dataset(
        space_id=ARIZE_SPACE_ID,
        dataset_name=TRAIN_DATASET_NAME,
        dataset_type=GENERATIVE,
        data=train_set,
    )

test_dataset_id = client.create_dataset(
        space_id=ARIZE_SPACE_ID,
        dataset_name=TEST_DATASET_NAME,
        dataset_type=GENERATIVE,
        data=test_set,
    )

# Get datasets as Dataframe
train_dataset = client.get_dataset(space_id=ARIZE_SPACE_ID, dataset_name=TRAIN_DATASET_NAME)
test_dataset = client.get_dataset(space_id=ARIZE_SPACE_ID, dataset_name=TEST_DATASET_NAME)

print("train dataset id:", train_dataset_id)
print("test dataset id:", test_dataset_id)

train dataset id: RGF0YXNldDozMjM1NjE6M0NOTg==
test dataset id: RGF0YXNldDozMjM1NjI6SXc5dA==


## **Base Prompt for Optimization**

This is our base prompt - our 0th iteration. This is the prompt we will be optimizing for our task.

We also upload our prompt to Arize AX. Arize's Prompt Hub serves as a repository for your prompts. You will be able to view all iterations of your prompt as its optimized, along with some metrics.

In [7]:
pip install -q "arize[PromptHub]"

Note: you may need to restart the kernel to use updated packages.


In [8]:
#from phoenix.client.types import PromptVersion
# Prompt Hub docs: https://arize.com/docs/ax/reference/reference/prompt-hub-api

from arize.experimental.prompt_hub import ArizePromptClient, Prompt, LLMProvider

# Initialize the client with your Arize credentials
prompt_client = ArizePromptClient(
    space_id=ARIZE_SPACE_ID,
    api_key=ARIZE_API_KEY
)


system_prompt = """
support query: {query}
Account Creation
Login Issues
Password Reset
Two-Factor Authentication
Profile Updates
Billing Inquiry
Refund Request
Subscription Upgrade/Downgrade
Payment Method Update
Invoice Request
Order Status
Shipping Delay
Product Return
Warranty Claim
Technical Bug Report
Feature Request
Integration Help
Data Export
Security Concern
Terms of Service Question
Privacy Policy Question
Compliance Inquiry
Accessibility Support
Language Support
Mobile App Issue
Desktop App Issue
Email Notifications
Marketing Preferences
Beta Program Enrollment
General Feedback

Return just the category, no other text.
"""


def add_prompt_version(system_prompt, prompt_name, model_name, test_metric, loop_number):
    try:
        existing_prompt = prompt_client.pull_prompt(prompt_name=prompt_name)
        existing_prompt.messages = [{"role": "system", "content": system_prompt}]
        existing_prompt.commit_message = f"Loop {loop_number} - Test Metric: {test_metric}"
        prompt_client.push_prompt(existing_prompt, commit_message = existing_prompt.commit_message)
    except:
        existing_prompt = Prompt(
            name=prompt_name,
            model_name=model_name,
            messages=[{"role": "system", "content": system_prompt}],
            provider=LLMProvider.OPENAI,
        )
        existing_prompt.commit_message = f"Loop {loop_number} \n Test Metric: {test_metric}"
        prompt_client.push_prompt(existing_prompt, commit_message = existing_prompt.commit_message)



/Users/gautamrangarajan/Desktop/workspace/prompt-learning/prompt_learning_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## **Output Generator**

This function calls OpenAI with our prompt on every row of our dataset to generate outputs. It leverages llm_generate, a Phoenix function, for concurrency in calling LLMs. 

We return the output column, which contains outputs for every row of our dataset, or every support query in our dataset. 

In [9]:
def generate_task(system_prompt):

    async def output_task(dataset_row):
        formatted_prompt = system_prompt.replace("{query}", dataset_row.get("query"))
        response = await openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": formatted_prompt}],
        )
        return response.choices[0].message.content
    
    return output_task

In [10]:
def normalize(label):
        return label.strip().strip('"').strip("'").lower()

## **Evaluator**

In this section we define our LLM-as-judge eval. 

Prompt Learning works by generating natural language evaluations on your outputs. These evaluations help guide the prompt optimizer towards building an optimized prompt. 

You should spend time thinking about how to write an informative eval. Your eval makes or breaks this prompt optimizer. With helpful feedback, our prompt optimizer will be able to generate a stronger optimized prompt much more effectively than with sparse or unhelpful feedback. 

Below is a great example for building a strong eval. You can see that we return many evaluations, including
- **correctness**: correct/incorrect - whether the support query was classified correctly or incorrectly.

-  **explanation**: Brief explanation of why the predicted classification is correct or incorrect, referencing the correct label if relevant.

-  **confusion_reason**: If incorrect, explains why the model may have made this choice instead of the correct classification. Focuses on likely sources of confusion. If correct, 'no confusion'.

-  **error_type**: One of: 'broad_vs_specific', 'keyword_bias', 'multi_intent_confusion', 'ambiguous_query', 'off_topic', 'paraphrase_gap', 'other'. Use 'none' if correct. Include the definition of the chosen error type, which are passed into the evaluator's prompt. 

-  **evidence_span**: Exact phrase(s) from the query that strongly indicate the correct classification.

-  **prompt_fix_suggestion**: One clear instruction to add to the classifier prompt to prevent this error.

**Take a look at support_query_classification/evaluator_prompt.txt for the full prompt!**

Our evaluator leverages llm_generate once again to build these llm evals with concurrency. We use an output parser to ensure that our eval is returned in proper json format. 

In [11]:
import re
#from phoenix.experiments.types import EvaluationResult
from arize.experimental.datasets.experiments.evaluators.base import EvaluationResult



def find_attributes(output):
    patterns = {
        "correctness": r'"correctness":\s*"([^"]*)"',
        "explanation": r'"explanation":\s*"([^"]*)"',
        "confusion_reason": r'"confusion_reason":\s*"([^"]*)"',
        "error_type": r'"error_type":\s*"([^"]*)"',
        "evidence_span": r'"evidence_span":\s*"([^"]*)"',
        "prompt_fix_suggestion": r'"prompt_fix_suggestion":\s*"([^"]*)"'
    }

    return tuple(
        (match := re.search(pattern, output, re.IGNORECASE)) and match.group(1)
        for pattern in patterns.values()
    )


def eval_parser(response: str) -> dict:
    correctness, explanation, confusion_reason, error_type, evidence_span, prompt_fix_suggestion = find_attributes(response)
    return {
        "correctness": correctness,
        "explanation": explanation,
        "confusion_reason": confusion_reason,
        "error_type": error_type,
        "evidence_span": evidence_span,
        "prompt_fix_suggestion": prompt_fix_suggestion
    }


async def output_evaluator(dataset_row, output):
    with open("../prompts/support_query_classification/evaluator_prompt.txt", "r") as file:
        evaluator_prompt = file.read()

    evaluator_prompt = evaluator_prompt.replace("{query}", dataset_row.get("query"))
    evaluator_prompt = evaluator_prompt.replace("{ground_truth}", dataset_row.get("ground_truth"))
    evaluator_prompt = evaluator_prompt.replace("{output}", output)

    eval_result = await openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": evaluator_prompt}],
        response_format={"type": "json_object"},
    )

    response = eval_result.choices[0].message.content
    parsed_eval_result = eval_parser(response)
    explanation=f"""correctness: {parsed_eval_result.get("correctness", "")};
        explanation: {parsed_eval_result.get("explanation", "")};
        confusion_reason: {parsed_eval_result.get("confusion_reason", "")};
        error_type: {parsed_eval_result.get("error_type", "")};
        evidence_span: {parsed_eval_result.get("evidence_span", "")};
        prompt_fix_suggestion: {parsed_eval_result.get("prompt_fix_suggestion", "")};"""

    score = float(parsed_eval_result.get("correctness") == "correct")
    label = parsed_eval_result.get("correctness", "")
    explanation = explanation

    return EvaluationResult(
        score=score,
        label=label,
        explanation=explanation,
    )

async def test_evaluator(dataset_row, output):
    label=str(normalize(dataset_row.get("ground_truth")) == normalize(output))
    return EvaluationResult(
        label=label,
        score = float(label=="True"),
        explanation="placeholder"
        )


## Metrics

Below we define some metrics that will compute on each iteration of prompt optimization. It will help us measure how our classifier with the current iteration's prompt performs.

Specifically we use scikit learn for precision, recall, f1 score, and simple accuracy.

In [12]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
def compute_metric(experiment_id, prediction_column_name, scorer="accuracy", average="macro"):
    """
    Compute the requested classification metric from a Arize experiment.
    """
    experiment_df = client.get_experiment(ARIZE_SPACE_ID, experiment_id=experiment_id)

    print(experiment_df.head())

    y_pred = experiment_df[prediction_column_name]
    y_true = [1]*len(experiment_df)

    if scorer == "accuracy":
        return accuracy_score(y_true, y_pred)
    elif scorer == "f1":
        return f1_score(y_true, y_pred, zero_division=0, average=average)
    elif scorer == "precision":
        return precision_score(y_true, y_pred, zero_division=0, average=average)
    elif scorer == "recall":
        return recall_score(y_true, y_pred, zero_division=0, average=average)
    else:
        raise ValueError(f"Unknown scorer: {scorer}")

## Experiment Processor

This function pulls an Arize experiment and loads the data into a pandas dataframe so it can run through the optimizer.

Specifically it:
- Pulls the experiment data from Arize
- Adds the input column to the dataframe
- Adds the evals to the dataframe
- Adds the output to the dataframe
- Returns the dataframe

In [13]:
def process_experiment(arize_client, experiment_id, train_set, input_column_name, output_column_name, feedback_columns = None):

    experiment_df = arize_client.get_experiment(ARIZE_SPACE_ID, experiment_id=experiment_id)

    train_set_with_experiment_results = pd.merge(train_set, experiment_df, left_on='id', right_on='example_id', how='inner')
    
    for column in feedback_columns:
        train_set[column] = [None] * len(train_set)
    
    for idx, row in train_set_with_experiment_results.iterrows():
        index = row["example_id"]
        eval_output = row["eval.output_evaluator.explanation"]
        if feedback_columns:
            for item in eval_output.split(";"):
                key_value = item.split(":")
                if key_value[0].strip() in feedback_columns:
                    key, value = key_value[0].strip(), key_value[1].strip()
                    train_set.loc[train_set["id"] == index, key] = value

            

    train_set[output_column_name] = train_set_with_experiment_results["result"]

    train_set.rename(columns={"query": input_column_name}, inplace=True)
    
    return train_set


# Optimization with Inbult Looping and Random Sampling
- This cell demonstrates use of the new optimize_with_experiments workflow for prompt optimization,
- including module reloading and running train/test experiments with full iterative evaluation and Prompt Hub integration.
- This new workflow is still in a testing phase and is expected to eventually replace the previous optimize method.



In [ ]:
import importlib
import optimizer_sdk.prompt_learning_optimizer

# Reload the module
importlib.reload(optimizer_sdk.prompt_learning_optimizer)

# Now re-import the class
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer

# Prompt name that will show up in Arize Prompt Hub
prompt_name = "prompt_optimizer+" + UUID

# Initialize the optimizer
optimizer = PromptLearningOptimizer(
    prompt=system_prompt,
    model_choice="gpt-4o",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Define feedback columns
feedback_columns = ["correctness", "explanation", "confusion_reason", "error_type", "evidence_span", "prompt_fix_suggestion"]

# Run the complete optimization workflow
optimized_prompt = optimizer.optimize_with_experiments(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    train_dataset_id=train_dataset_id,
    test_dataset_id=test_dataset_id,
    evaluators=[output_evaluator],
    test_evaluator=test_evaluator,
    task_fn=generate_task,
    arize_client=client,
    arize_space_id=ARIZE_SPACE_ID,
    experiment_name_prefix=f"optimization_{UUID}_3",
    prompt_hub_client=prompt_client,
    prompt_name=prompt_name,
    model_name="gpt-4o-mini-2024-07-18",
    feedback_columns=feedback_columns,
    threshold=1.0,
    loops=5,
    scorer="accuracy",
    context_size_k=5000,
)

🚀 Starting prompt optimization with 2 iterations (scorer: accuracy, threshold: 1.0)

📊 Initial evaluation:
⏳ Running initial evaluation experiment...
  arize.utils.logging | INFO | 🧪 Experiment started.


running tasks |          | 0/31 (0.0%) | ⏳ 00:00<? | ?it/s

  arize.utils.logging | INFO | ✅ Task runs completed.
Tasks Summary (10/19/25 11:55 PM -0400)
---------------------------------------
   n_examples  n_runs  n_errors
0          31      31         0


running experiment evaluations |          | 0/31 (0.0%) | ⏳ 00:00<? | ?it/s

  arize.utils.logging | INFO | ✅ All evaluators completed.
⏳ Retrieving experiment results...
✅ Initial accuracy: 0.548
⏳ Uploading initial prompt version to Prompt Hub...
✅ Uploaded to Prompt Hub
🔧 Calculated max rows for context: 19 rows fit within 1,000 tokens
🔢 Maximum number of rows for training context: 19
🔧 EFFICIENCY SETUP: Will sample 19 examples from 123 total training examples each loop
   💰 This saves ~104 evaluator calls per loop!

📊 Loop 1: Optimizing prompt...
   🎲 Sampled 19 examples for this loop
⏳ Running training experiment...
  arize.utils.logging | INFO | 🧪 Experiment started.


...

  arize.utils.logging | INFO | ✅ All evaluators completed.
⏳ Retrieving training experiment results...
✅ Training accuracy: 0.526
⏳ Processing experiment results and extracting feedback...
⏳ Optimizing prompt with meta-prompt...
✅ Prompt optimization complete
⏳ Running test evaluation experiment...
  arize.utils.logging | INFO | 🧪 Experiment started.


running tasks |          | 0/31 (0.0%) | ⏳ 00:00<? | ?it/s

  arize.utils.logging | INFO | ✅ Task runs completed.
Tasks Summary (10/19/25 11:58 PM -0400)
---------------------------------------
   n_examples  n_runs  n_errors
0          31      31         0


running experiment evaluations |          | 0/31 (0.0%) | ⏳ 00:00<? | ?it/s

  arize.utils.logging | INFO | ✅ All evaluators completed.
⏳ Retrieving test experiment results...
✅ Test accuracy: 0.581
⏳ Uploading prompt version to Prompt Hub...
✅ Uploaded to Prompt Hub


╔════════════════════════════════════════════════════════════════════════════════╗
║                      PROMPT OPTIMIZATION SUMMARY                               ║
╚════════════════════════════════════════════════════════════════════════════════╝

Optimization completed after 2 iteration(s)
Final test accuracy: 0.581
Improvement over baseline: +5.9%

┌──────┬─────────────┬─────────────┬──────────────────┬──────────────────┐
│ Loop │ Train Acc   │ Test Acc    │ Train Exp ID     │ Test Exp ID      │
├──────┼─────────────┼─────────────┼──────────────────┼──────────────────┤
│   0  │      -      │   0.548     │        -         │ RXhwZXJpbWVud... │
│   1  │   0.737     │   0.581     │ RXhwZXJpbWVud... │ RXhwZXJpbWVud... │
│   2  │   0.526     │   0.581     │ RXhwZXJpbWVud... │ RXhwZXJpbWVud... │
└─

# Prompt Optimization Loop with Arize Experiments

This code implements an iterative prompt optimization system that uses **Arize AX experiments** to evaluate and improve prompts based on feedback from LLM evaluators.


## Overview

The `optimize_loop` function automates prompt engineering by:

- Evaluating prompts using Arize experiments  
- Collecting detailed feedback from LLM evaluators  
- Optimizing prompts via a learning-based optimizer  
- Iterating until the performance threshold is met or the loop limit is reached  


## Step-by-Step Breakdown

Each of these numbers are added as comments in the code.

### 1. Initialization

- Set up tracking variables:
  - `train_metrics`, `test_metrics`, `raw_dfs` for storing evaluation results
- Convert training dataset to a DataFrame for easy updates

### 2. Baseline Evaluation

- Run an initial experiment using the **test set**
- Establish a **baseline metric** (e.g., accuracy, F1) to compare against future improvements

### 3. Early Exit Check

- If the **initial prompt already meets the performance threshold**, skip further optimization to save time and compute

### 4. Main Optimization Loop

For each iteration (up to `loops`):

#### 4a. Run Training Experiment

- Execute the current prompt on the **training set**
- Use LLM evaluators to generate **natural language feedback**

#### 4b. Process Feedback

- Extract structured information from evaluator outputs:
  - Correctness
  - Explanation
  - Confusion reason
  - Error type
  - Prompt fix suggestions
- Update the training DataFrame with this feedback

#### 4c. Generate Learning Annotations

- Convert feedback into structured annotations for the optimizer to learn from
- This allows learning from evaluator insights in a consistent format

#### 4d. Optimize the Prompt

- Pass feedback to the **PromptLearningOptimizer**
- Generate an **improved prompt** that attempts to correct issues found in the previous iteration

#### 4e. Evaluate on Test Set

- Evaluate the updated prompt on the **held-out test set**
- Assess **generalization** beyond the training data

#### 4f. Track Metrics

- Log metrics for:
  - Training set performance
  - Test set performance
- Store raw results for further analysis or visualization

#### 4g. Convergence Check

- If the new prompt's test metric **meets or exceeds the threshold**, exit the loop early

In [ ]:
from optimizer_sdk.prompt_learning_optimizer import PromptLearningOptimizer
import time

import copy

#prompt name that will show up in Arize Prompt Hub
prompt_name = "prompt_optimizer+" + UUID

def optimize_loop(
    train_dataset,
    test_dataset,
    system_prompt,
    feedback_columns,
    threshold=1,
    loops=5,
    scorer="accuracy",
    prompt_versions=[],
):
    """
    scorer: one of "accuracy", "f1", "precision", "recall"
    """
    curr_loop = 1
    train_metrics = []
    test_metrics = []
    raw_dfs = []
    train_df = train_dataset

    print(f"🚀 Starting prompt optimization with {loops} iterations (scorer: {scorer}, threshold: {threshold})")
    
    print(f"�� Initial evaluation:")

    task = generate_task(system_prompt)

    initial_experiment_id, _ = client.run_experiment(
        space_id=ARIZE_SPACE_ID,
        dataset_id=test_dataset_id,
        task=task,
        evaluators=[test_evaluator],
        experiment_name="initial_experiment_1",
        concurrency=10
    )

    time.sleep(3)
    
    initial_metric_value = compute_metric(initial_experiment_id, "eval.test_evaluator.score", scorer)
    print(f"✅ Initial {scorer}: {initial_metric_value}")

    test_metrics.append(initial_metric_value)
    raw_dfs.append(copy.deepcopy(test_set))
    prompt_versions.append(system_prompt)

    add_prompt_version(system_prompt, prompt_name, "gpt-4o-mini-2024-07-18", initial_metric_value, 0)
    if initial_metric_value >= threshold:
        print("🎉 Initial prompt already meets threshold!")
        return {
            "train": train_metrics,
            "test": test_metrics,
            "prompt": prompt_versions,
            "raw": raw_dfs
        }
    
    # Initialize all feedback columns

    while loops > 0:
        print(f"📊 Loop {curr_loop}: Optimizing prompt...")
        
        task = generate_task(system_prompt)

        train_experiment_id, _ = client.run_experiment(
            space_id=ARIZE_SPACE_ID,
            dataset_id=train_dataset_id,
            task=task,
            evaluators=[output_evaluator, test_evaluator],
            experiment_name=f"train_experiment_{curr_loop}",
            concurrency=10
        )

        time.sleep(3)

        train_metric = compute_metric(train_experiment_id, "eval.output_evaluator.score", "accuracy")
        train_metrics.append(train_metric)
        train_df = process_experiment(client, train_experiment_id, train_df, "query", "output", feedback_columns)
        print(f"✅ Training {scorer}: {train_metric}")
        
        optimizer = PromptLearningOptimizer(
            prompt=system_prompt,  
            model_choice="gpt-4o",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )

        with open("../prompts/support_query_classification/annotations_prompt.txt", "r") as file:
            annotations_prompt = file.read()

        annotations = optimizer.create_annotation(
            system_prompt,
            ["query"],
            train_df,
            feedback_columns,
            [annotations_prompt],
            "output",
            "ground_truth"
        )

        system_prompt = optimizer.optimize(
            train_df,
            "output",
            feedback_columns=feedback_columns,
            context_size_k=90000,
            annotations=annotations,
        )
        prompt_versions.append(system_prompt)

        
        test_experiment_id, _ = client.run_experiment(
            space_id=ARIZE_SPACE_ID,
            dataset_id=test_dataset_id,
            task=generate_task(system_prompt),
            evaluators=[test_evaluator],
            experiment_name=f"test_experiment_{curr_loop}",
            concurrency=10
        )

        time.sleep(3)

        test_metric = compute_metric(test_experiment_id, "eval.test_evaluator.score", scorer)
        test_metrics.append(test_metric)

        add_prompt_version(system_prompt, prompt_name, "gpt-4o-mini-2024-07-18", test_metric, curr_loop)
        print(f"✅ Test {scorer}: {test_metric}")

        if test_metric >= threshold:
            print("🎉 Prompt optimization met threshold!")
            break

        loops -= 1
        curr_loop += 1

    return {
        "train": train_metrics,
        "test": test_metrics,
        "prompts": prompt_versions,
        "raw": raw_dfs
    }

# Main execution - use asyncio.run() to run the async function
evaluators = [output_evaluator]
feedback_columns = ["correctness", "explanation", "confusion_reason", "error_type", "evidence_span", "prompt_fix_suggestion"]
result = optimize_loop(train_dataset, test_dataset, system_prompt, feedback_columns, loops=5, scorer="accuracy")

# Prompt Optimized!

The code below picks the prompt with the highest score on the test set, and displays the training/test metrics and delta for that prompt.

In [ ]:
# Find the best index based on highest test accuracy
best_idx = max(range(len(result["test"])), key=lambda i: result["test"][i])

# Retrieve values
best_prompt = result["prompts"][best_idx - 1]
best_test_acc = result["test"][best_idx]
best_train_acc = result["train"][best_idx - 1] if (best_idx - 1) < len(result["train"]) else None
initial_test_acc = result["test"][0]
initial_train_acc = result["train"][0] if result["train"] else None

# Print results
print("\n🔍 Best Prompt Found:")
print(best_prompt)
print(f"🧪 Initial Test Accuracy: {initial_test_acc}")
print(f"🧪 Optimized Test Accuracy: {best_test_acc} (Δ {best_test_acc - initial_test_acc:.4f})")